In [1]:
import sys
from Load import *
from UE import *
from Satellite import *
from Oracle import *
import time


def initial_assignment(UEss, Satss, oracle):
    C = UEss[0].coverage_info
    if oracle is None:
        # random assign
        for ueid in UEss:
            ue = UEss[ueid]
            possible_satellites = np.where(C[ueid, :, 0] == 1)[0]
            assert (len(possible_satellites) > 0)
            satid = random.choice(possible_satellites)
            ue.serving_satellite = Satss[satid]
            ue.serving_satellite_history.append(satid)
    else:
        for ueid in UEss:
            ue = UEss[ueid]
            satid = oracle.query_init_satellite(ueid)
            assert (C[ueid, satid, 0] == 1)
            ue.serving_satellite = Satss[satid]
            ue.serving_satellite_history.append(satid)


def monitor_timestamp(env):
    while True:
        if env.now % 100 == 0:
            print(f"Current simulation time {env.now}", file=sys.stderr)
        yield env.timeout(1)


if __name__ == "__main__":
    # configuration
    # When oracle_simulation is False, the oracle_assignment could be True/False
    # When oracle_simulation is True, the oracle_assignment must be True
    max_access_opportunity = 1
    max_access_slots = 100
    oracle_assignment = False
    oracle_simulation = False
    if oracle_simulation:
        assert (oracle_assignment)
    random.seed(10)

    # Loading scenarios
    beginning = time.time()
    UEs_template, satellites_template, C = load_scenario()
    DURATION = C.shape[2]
    env = simpy.Environment()

    assignment_oracle = None
    simulation_oracle = None
    if oracle_assignment:
        assignment_oracle = Oracle()
    if oracle_simulation:
        simulation_oracle = assignment_oracle

    UEs = {}
    for ue_template in UEs_template:
        ID = ue_template.ID
        if ID not in UEs:
            UEs[ID] = UE(
                identity=ID,
                position_x=ue_template.x,
                position_y=ue_template.y,
                coverage_info=C,
                oracle=simulation_oracle,
                env=env
            )
        else:
            print("ERROR")

    Satellites = {}
    for sat_template in satellites_template:
        ID = sat_template.ID
        if ID not in Satellites:
            Satellites[ID] = Satellite(
                identity=ID,
                position_x=sat_template.x,
                position_y=sat_template.y,
                height=sat_template.h,
                coverage_r=sat_template.r,
                velocity=sat_template.v,
                sind=sat_template.sind,
                cosd=sat_template.cosd,
                coverage_info=C,
                max_access_opportunity = max_access_opportunity,
                max_access_slots = max_access_slots,
                env=env
            )
        else:
            print("ERROR")

    for ueid in UEs:
        UEs[ueid].satellites = Satellites

    for satid in Satellites:
        Satellites[satid].UEs = UEs
        Satellites[satid].satellites = Satellites

    print(f"Loading scenario in the simulation takes {time.time() - beginning}")
    # ========= UE, SAT objectives are ready =========
    initial_assignment(UEs, Satellites, assignment_oracle)

    env.process(monitor_timestamp(env))

    print('============= Experiment Log =============')
    env.run(until=DURATION)
    print('============= Experiment Ends =============')
    counter = allCounters(Satellites, UEs)
    counter.generate_heap_map(100)
    counter.generate_delay_box()


Loading scenario in the simulation takes 0.06910324096679688
============= Experiment Log =============
UE 37 sends Satellite 1 the message {"task": "MEASUREMENT_REPORT", "candidates": [2, 3, 4], "from": 37, "to": 1} at 199
Satellite 1 sends Satellite 2 the message {"task": "HANDOVER_REQUEST", "ueid": 37, "from": 1, "to": 2} at 199
Satellite 1 sends Satellite 3 the message {"task": "HANDOVER_REQUEST", "ueid": 37, "from": 1, "to": 3} at 199
Satellite 1 sends Satellite 4 the message {"task": "HANDOVER_REQUEST", "ueid": 37, "from": 1, "to": 4} at 199
Satellite 2 sends Satellite 1 the message {"task": "HANDOVER_RESPONSE", "ueid": 37, "condition": {"access_delay": 1, "ueid": 37, "source": 1, "satid": 2}, "from": 2, "to": 1} at 199
Satellite 3 sends Satellite 1 the message {"task": "HANDOVER_RESPONSE", "ueid": 37, "condition": {"access_delay": 1, "ueid": 37, "source": 1, "satid": 3}, "from": 3, "to": 1} at 199
Satellite 4 sends Satellite 1 the message {"task": "HANDOVER_RESPONSE", "ueid": 37

Current simulation time 0
Current simulation time 100
Current simulation time 200
Current simulation time 300
Current simulation time 400
Current simulation time 500
Current simulation time 600
Current simulation time 700
Current simulation time 800
Current simulation time 900
Current simulation time 1000
Current simulation time 1100
Current simulation time 1200
Current simulation time 1300
Current simulation time 1400
Current simulation time 1500
Current simulation time 1600
Current simulation time 1700
Current simulation time 1800


UE 3 sends Satellite 2 the message {"task": "MEASUREMENT_REPORT", "candidates": [3, 4], "from": 3, "to": 2} at 1747
Satellite 2 sends Satellite 3 the message {"task": "HANDOVER_REQUEST", "ueid": 3, "from": 2, "to": 3} at 1747
Satellite 2 sends Satellite 4 the message {"task": "HANDOVER_REQUEST", "ueid": 3, "from": 2, "to": 4} at 1747
Satellite 3 sends Satellite 2 the message {"task": "HANDOVER_RESPONSE", "ueid": 3, "condition": {"access_delay": 1, "ueid": 3, "source": 2, "satid": 3}, "from": 3, "to": 2} at 1747
Satellite 4 sends Satellite 2 the message {"task": "HANDOVER_RESPONSE", "ueid": 3, "condition": {"access_delay": 1, "ueid": 3, "source": 2, "satid": 4}, "from": 4, "to": 2} at 1747
Satellite 2 sends UE 3 the message {"task": "RRC_RECONFIGURATION", "conditions": [{"access_delay": 1, "ueid": 3, "source": 2, "satid": 3}, {"access_delay": 1, "ueid": 3, "source": 2, "satid": 4}], "from": 2, "to": 3} at 1747
UE 3 sends Satellite 2 the message {"task": "RRC_RECONFIGURATION_COMPLETE", "

Current simulation time 1900
Current simulation time 2000
Current simulation time 2100
Current simulation time 2200
Current simulation time 2300
Current simulation time 2400
Current simulation time 2500
Current simulation time 2600
Current simulation time 2700
Current simulation time 2800
Current simulation time 2900
Current simulation time 3000
Current simulation time 3100
Current simulation time 3200
Current simulation time 3300
Current simulation time 3400
Current simulation time 3500
Current simulation time 3600
Current simulation time 3700
Current simulation time 3800
Current simulation time 3900


UE 29 sends Satellite 0 the message {"task": "MEASUREMENT_REPORT", "candidates": [1, 2], "from": 29, "to": 0} at 3481
Satellite 0 sends Satellite 1 the message {"task": "HANDOVER_REQUEST", "ueid": 29, "from": 0, "to": 1} at 3481
Satellite 0 sends Satellite 2 the message {"task": "HANDOVER_REQUEST", "ueid": 29, "from": 0, "to": 2} at 3481
Satellite 1 sends Satellite 0 the message {"task": "HANDOVER_RESPONSE", "ueid": 29, "condition": {"access_delay": 1, "ueid": 29, "source": 0, "satid": 1}, "from": 1, "to": 0} at 3481
Satellite 2 sends Satellite 0 the message {"task": "HANDOVER_RESPONSE", "ueid": 29, "condition": {"access_delay": 1, "ueid": 29, "source": 0, "satid": 2}, "from": 2, "to": 0} at 3481
Satellite 0 sends UE 29 the message {"task": "RRC_RECONFIGURATION", "conditions": [{"access_delay": 1, "ueid": 29, "source": 0, "satid": 1}, {"access_delay": 1, "ueid": 29, "source": 0, "satid": 2}], "from": 0, "to": 29} at 3481
UE 29 sends Satellite 0 the message {"task": "RRC_RECONFIGURATION

...

UE 9 sends Satellite 9 the message {"task": "MEASUREMENT_REPORT", "candidates": [10, 11, 12], "from": 9, "to": 9} at 16122
Satellite 9 sends Satellite 10 the message {"task": "HANDOVER_REQUEST", "ueid": 9, "from": 9, "to": 10} at 16122
Satellite 9 sends Satellite 11 the message {"task": "HANDOVER_REQUEST", "ueid": 9, "from": 9, "to": 11} at 16122
Satellite 9 sends Satellite 12 the message {"task": "HANDOVER_REQUEST", "ueid": 9, "from": 9, "to": 12} at 16122
Satellite 10 sends Satellite 9 the message {"task": "HANDOVER_RESPONSE", "ueid": 9, "condition": {"access_delay": 1, "ueid": 9, "source": 9, "satid": 10}, "from": 10, "to": 9} at 16122
Satellite 11 sends Satellite 9 the message {"task": "HANDOVER_RESPONSE", "ueid": 9, "condition": {"access_delay": 1, "ueid": 9, "source": 9, "satid": 11}, "from": 11, "to": 9} at 16122
Satellite 12 sends Satellite 9 the message {"task": "HANDOVER_RESPONSE", "ueid": 9, "condition": {"access_delay": 1, "ueid": 9, "source": 9, "satid": 12}, "from": 12, "

Current simulation time 17500
Current simulation time 17600
Current simulation time 17700
Current simulation time 17800
Current simulation time 17900
Current simulation time 18000
Current simulation time 18100
Current simulation time 18200
Current simulation time 18300
Current simulation time 18400
Current simulation time 18500
Current simulation time 18600
Current simulation time 18700
Current simulation time 18800
Current simulation time 18900
Current simulation time 19000
Current simulation time 19100
Current simulation time 19200
Current simulation time 19300
Current simulation time 19400
Current simulation time 19500
Current simulation time 19600


UE 44 sends Satellite 9 the message {"task": "MEASUREMENT_REPORT", "candidates": [10, 11, 12], "from": 44, "to": 9} at 18275
Satellite 9 sends Satellite 10 the message {"task": "HANDOVER_REQUEST", "ueid": 44, "from": 9, "to": 10} at 18275
Satellite 9 sends Satellite 11 the message {"task": "HANDOVER_REQUEST", "ueid": 44, "from": 9, "to": 11} at 18275
Satellite 9 sends Satellite 12 the message {"task": "HANDOVER_REQUEST", "ueid": 44, "from": 9, "to": 12} at 18275
Satellite 10 sends Satellite 9 the message {"task": "HANDOVER_RESPONSE", "ueid": 44, "condition": {"access_delay": 1, "ueid": 44, "source": 9, "satid": 10}, "from": 10, "to": 9} at 18275
Satellite 11 sends Satellite 9 the message {"task": "HANDOVER_RESPONSE", "ueid": 44, "condition": {"access_delay": 1, "ueid": 44, "source": 9, "satid": 11}, "from": 11, "to": 9} at 18275
Satellite 12 sends Satellite 9 the message {"task": "HANDOVER_RESPONSE", "ueid": 44, "condition": {"access_delay": 1, "ueid": 44, "source": 9, "satid": 12}, "f

Current simulation time 19700
Current simulation time 19800
Current simulation time 19900


UE 47 sends Satellite 11 the message {"task": "MEASUREMENT_REPORT", "candidates": [12, 13, 14], "from": 47, "to": 11} at 19794
Satellite 11 sends Satellite 12 the message {"task": "HANDOVER_REQUEST", "ueid": 47, "from": 11, "to": 12} at 19794
Satellite 11 sends Satellite 13 the message {"task": "HANDOVER_REQUEST", "ueid": 47, "from": 11, "to": 13} at 19794
Satellite 11 sends Satellite 14 the message {"task": "HANDOVER_REQUEST", "ueid": 47, "from": 11, "to": 14} at 19794
Satellite 12 sends Satellite 11 the message {"task": "HANDOVER_RESPONSE", "ueid": 47, "condition": {"access_delay": 1, "ueid": 47, "source": 11, "satid": 12}, "from": 12, "to": 11} at 19794
Satellite 13 sends Satellite 11 the message {"task": "HANDOVER_RESPONSE", "ueid": 47, "condition": {"access_delay": 1, "ueid": 47, "source": 11, "satid": 13}, "from": 13, "to": 11} at 19794
Satellite 14 sends Satellite 11 the message {"task": "HANDOVER_RESPONSE", "ueid": 47, "condition": {"access_delay": 1, "ueid": 47, "source": 11, 

In [2]:
from collections import deque

# Assuming 'data' is your list of lists, converted to a deque
data = deque([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

In [3]:
len(data)

3

In [7]:
0.9999999 * 20000

19999.998

In [ ]:
data.popleft

In [6]:
data.remove([1, 2, 3])

In [7]:
data

deque([[4, 5, 6], [7, 8, 9]])

In [34]:
import random
s = time.time()
10000+ 20000 
(time.time() - s)*1000


0.1430511474609375

In [4]:
Satellites[0].access_Q.max_opportunity

0

In [10]:
import timeit

# List comprehension timing
time_list_comprehension = timeit.timeit(
    stmt='[index for index, value in enumerate(lst) if value > 0]',
    setup='lst = [0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1]',
    number=100
)

# Loop timing
time_loop = timeit.timeit(
    stmt='''
indexes = []
for index, value in enumerate(lst):
    if value > 0:
        indexes.append(index)
''',
    setup='lst = [0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1]',
    number=100
)

(time_list_comprehension, time_loop)


(0.0004507080000166752, 0.00042033400001173504)